In [98]:
import pandas as pd
from dijkstar import Graph, find_path

In [99]:
mdf = pd.read_csv('trajets_tgv.csv',delimiter=',', error_bad_lines=False)
mdf.head()

,source,target,weight,annee,mois,commentaire
0,TOULOUSE MATABIAU,PARIS MONTPARNASSE,89.0,2015,2,Cette destination a été touchée par 6 incident...
1,VANNES,PARIS MONTPARNASSE,90.7,2015,2,Cette ligne a été touchée par un incident maje...
2,LA ROCHELLE VILLE,PARIS MONTPARNASSE,94.1,2015,3,Cette OD a été touchée par les événements suiv...
3,ST PIERRE DES CORPS,PARIS MONTPARNASSE,84.3,2015,2,NaN
4,LE CREUSOT MONTCEAU MONTCHANIN,PARIS LYON,94.9,2015,3,NaN


In [100]:

def get_reg(df, a, b):
    tmp = df[df['source'] == a]
    return tmp[tmp['target'] == b]['weight'].as_matrix()[0]
def get_com(df, a, b):
    tmp = df[df['source'] == a]
    return tmp[tmp['target'] == b]['commentaire'].as_matrix()[0]


In [109]:
from functools import reduce
list_ = []
stations = set(df['source'])
links = df[['source', 'target']].as_matrix()
graph = Graph()
for a, b in links:
    graph.add_edge(a, b, {'cost': 1})
cost_func = lambda u, v, e, prev_e: e['cost']
        
for year in range(2011, 2018):
    for mois in range(1, 13):
        tmp = mdf[mdf['annee'] == year]
        df = tmp[tmp['mois'] == mois]

        stations = set(df['source'])
        links = df[['source', 'target']].as_matrix()
        graph = Graph()
        for a, b in links:
            graph.add_edge(a, b, {'cost': 1})
        cost_func = lambda u, v, e, prev_e: e['cost']
        
        for a in stations:
            for b in stations:
                if a != b:
                    nodes = find_path(graph, a, b, cost_func=cost_func).nodes
                    depart = nodes[0]
                    arrive = nodes[-1]
                    regularite = reduce(lambda a, b: a*b/100, [get_reg(df, nodes[i], nodes[i + 1]) for i in range(len(nodes) - 1)], 1) 
                    commentaire = ""
                    list_.append(
                        [a, b, year, mois, regularite, commentaire]
                    )

In [110]:
import numpy
a = numpy.asarray(list_)
d = pd.DataFrame(a)
d.to_csv("multiple.csv", sep=',', header=['source', 'target', 'annee', 'mois', 'weight', 'commentaire'])


In [128]:
df = pd.read_csv('../../projet/data/multiple.csv').dropna(axis=1, how='all')
df.head()
df.to_csv("../../projet/data/multiple2.csv", sep=',', header=['', 'toto', 'source', 'target', 'weight', 'annee', 'mois', 'commentaire'])


In [88]:
tmp = df[df['Départ'] == 'VANNES']
tmp[tmp['Arrivée']=='PARIS MONTPARNASSE']

nodes = find_path(graph, 'VANNES', 'PARIS MONTPARNASSE', cost_func=cost_func).nodes
date = year
depart = nodes[0]
arrive = nodes[-1]
print('a', type(get_reg(df, 'VANNES', 'PARIS MONTPARNASSE')))
regularite = reduce(lambda a, b: a*b/100, [get_reg(df, nodes[i], nodes[i + 1]) for i in range(len(nodes) - 1)], 1) 
print(
    {
        'Départ':'VANNES', 
        'Arrivée':'PARIS MONTPARNASSE', 
        'Date': date, 
        'Regularité': regularite
    }
)

a <class 'numpy.float64'>
{'Départ': 'VANNES', 'Arrivée': 'PARIS MONTPARNASSE', 'Date': '4', 'Regularité': 0.92599999999999993}
